In [1]:
# h1 초기 해시값 구하기 

In [1]:
import math
import numpy as np

n = math.sqrt(3) - 1
inx = np.zeros((8, 4))

for i in range(32):
    row = i//4
    remain = i%4
    
    n = n*2
    print(i+1, " - ", n/2, " * 2 = ", n)
    
    if n < 1:
        inx[row][remain] = 0
    else:
        n = n - 1
        inx[row][remain] = 1

inx = inx.flatten()
bit = ""
for i in inx:
    bit += str(int(i))
    
bit = "0b" + bit
n_10 = int(bit, 2)
n_16 = hex(n_10)
print(n_16)

1  -  0.7320508075688772  * 2 =  1.4641016151377544
2  -  0.4641016151377544  * 2 =  0.9282032302755088
3  -  0.9282032302755088  * 2 =  1.8564064605510175
4  -  0.8564064605510175  * 2 =  1.712812921102035
5  -  0.7128129211020351  * 2 =  1.4256258422040702
6  -  0.4256258422040702  * 2 =  0.8512516844081404
7  -  0.8512516844081404  * 2 =  1.7025033688162807
8  -  0.7025033688162807  * 2 =  1.4050067376325615
9  -  0.40500673763256145  * 2 =  0.8100134752651229
10  -  0.8100134752651229  * 2 =  1.6200269505302458
11  -  0.6200269505302458  * 2 =  1.2400539010604916
12  -  0.24005390106049163  * 2 =  0.48010780212098325
13  -  0.48010780212098325  * 2 =  0.9602156042419665
14  -  0.9602156042419665  * 2 =  1.920431208483933
15  -  0.920431208483933  * 2 =  1.840862416967866
16  -  0.840862416967866  * 2 =  1.681724833935732
17  -  0.681724833935732  * 2 =  1.363449667871464
18  -  0.36344966787146404  * 2 =  0.7268993357429281
19  -  0.7268993357429281  * 2 =  1.4537986714858562
20  -

In [ ]:
# SHA256 해시 작동 원리

In [83]:
from bitstring import BitArray

In [84]:
# hash constants K[0] ~ k[63]
constants = (
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
            0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
            0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
            0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
            0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
            0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
            0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
            0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
            0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2)
# h0 ~ h7
h = (
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19)

In [85]:
def Pad(W):
    return bytes(W, "ascii") + b"\x80" + (b"\x00" * ((55 if (len(W) % 64) < 56 else 119) - (len(W) % 64))) + (
        (len(W) << 3).to_bytes(8, "big"))

In [86]:
message = input("Enter string : ")
message = Pad(message) # byte 
print("Padding : \n",BitArray(message).bin) # 바이트를 비트로 

Enter string : Abc
Padding : 
 01000001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000


In [87]:
def right_rotate(num, shift):
    # x에서 b 비트만큼 오른쪽으로 이동 
    return ((num >> shift) | (num << (32 - shift)))

In [88]:
# 64개의 message array W[0] ~ W[63] 만들기
# 512 bit == 64 byte 
for i in range(0, len(message), 64):
    # 64바이트 가져오기 == 512비트 가져오기
    S = message[i: i + 64]
    
    # 32비트씩 나누기 => 16개로 나누기 W[0] ~ W[15]
    W = [int.from_bytes(S[e: e + 4], 'big') for e in range(0, 64, 4)] + ([0] * 48)
    
    # W[16] ~ W[63]
    for j in range(16, 64):
        sig0 = (right_rotate(W[j-15], 7) ^ right_rotate(W[j-15], 18) ^ (W[j-15] >> 3))
        sig1 = (right_rotate(W[j-2], 17) ^ right_rotate(W[j-2], 19) ^ (W[j-2] >> 10))
        W[j] = (((W[j-16] + sig0 + W[j-7] + sig1))) & ((2 ** 32) - 1)
        
for idx, w in enumerate(W):
    print("W[{}] = {:032b}".format(idx, w))



W[0] = 01000001011000100110001110000000
W[1] = 00000000000000000000000000000000
W[2] = 00000000000000000000000000000000
W[3] = 00000000000000000000000000000000
W[4] = 00000000000000000000000000000000
W[5] = 00000000000000000000000000000000
W[6] = 00000000000000000000000000000000
W[7] = 00000000000000000000000000000000
W[8] = 00000000000000000000000000000000
W[9] = 00000000000000000000000000000000
W[10] = 00000000000000000000000000000000
W[11] = 00000000000000000000000000000000
W[12] = 00000000000000000000000000000000
W[13] = 00000000000000000000000000000000
W[14] = 00000000000000000000000000000000
W[15] = 00000000000000000000000000011000
W[16] = 01000001011000100110001110000000
W[17] = 00000000000011110000000000000000
W[18] = 01111101101000000111000000000101
W[19] = 01100000000000000000001111000110
W[20] = 00110110000111010111100101111000
W[21] = 00000001100000111111110000000000
W[22] = 00010011100111101001101010101011
W[23] = 11000010111000101100001110001110
W[24] = 100111100001001100

In [89]:
# initialize working variables to current has values
digest = list(h)
A, B, C, D, E, F, G, H = digest
print("A : {:032b} \nB : {:032b} \nC : {:032b} \nD : {:032b} \nE : {:032b} \nF : {:032b} \nG : {:032b}\nH : {:032b}\n".format(A, B, C, D, E, F, G, H))

A : 01101010000010011110011001100111 
B : 10111011011001111010111010000101 
C : 00111100011011101111001101110010 
D : 10100101010011111111010100111010 
E : 01010001000011100101001001111111 
F : 10011011000001010110100010001100 
G : 00011111100000111101100110101011
H : 01011011111000001100110100011001



In [90]:
# compression
def compression(Wt, Kt, A, B, C, D, E, F, G, H):
    sigma0 = (right_rotate(A, 2) ^ right_rotate(A, 13) ^ right_rotate(A, 22)) % 2**32
    sigma1 = (right_rotate(E, 6) ^ right_rotate(E, 11) ^ right_rotate(E, 25)) % 2**32
    ch = (E & F) ^ (~E & G)
    tmp1 = (H + sigma1 + ch + Kt + Wt) % 2**32
    maj = (A & B) ^ (A & C) ^ (B & C)
    tmp2 = (sigma0 + maj) % 2**32
    H = G
    G = F
    F = E
    E = (D + tmp1) % 2**32
    D = C
    C = B
    B = A
    A = (tmp1 + tmp2) % 2**32
    print("sigma0 : {:032b} \nsigma1 : {:032b} \nch : {:032b} \ntmp1 : {:032b} \nmaj : {:032b} \ntmp2 : {:032b} \n".format(sigma0, sigma1, ch, tmp1, maj, tmp2))
    return A, B, C, D, E, F, G, H

In [91]:
for i in range(64):
    A, B, C, D, E, F, G, H = compression(W[i], constants[i], A, B, C, D, E, F, G, H)
    print(f"======================= {i} th =======================")
    print("A : {:032b} \nB : {:032b} \nC : {:032b} \nD : {:032b} \nE : {:032b} \nF : {:032b} \nG : {:032b}\nH : {:032b}\n".format(A, B, C, D, E, F, G, H))
    print("=====================================================")

sigma0 : 11001110001000001011010001111110 
sigma1 : 00110101100001110010011100101011 
ch : 00011111100001011100100110001100 
tmp1 : 00110100110110100101000011101000 
maj : 00111010011011111110011001100111 
tmp2 : 00001000100100001001101011100101 

======================= 0 th =======================
A : 00111101011010101110101111001101 
B : 01101010000010011110011001100111 
C : 10111011011001111010111010000101 
D : 00111100011011101111001101110010 
E : 11011010001010100100011000100010 
F : 01010001000011100101001001111111 
G : 10011011000001010110100010001100
H : 00011111100000111101100110101011

sigma0 : 10111010100111000110010101010001 
sigma1 : 01011010000100001111110100111101 
ch : 01010001000011110110101010101110 
tmp1 : 00111011110110111000011000100111 
maj : 00111011011010111110111011000101 
tmp2 : 11110110000010000101010000010110 

======================= 1 th =======================
A : 00110001111000111101101000111101 
B : 00111101011010101110101111001101 
C : 011010100000100

In [25]:
compressed_chunk = [(x + y) % 2**32 for x, y in zip(digest, (A, B, C, D, E, F, G, H))]

In [26]:
test = [item.to_bytes(4, 'big') for item in compressed_chunk]
test2 = b"".join(test)
"".join(format(h, '02x') for h in test2)

'06d90109c8cce34ec0c776950465421e176f08b831a938b3c6e76cb7bee8790b'